In [1]:
import os
import yfinance as yf
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
# from langchain.llms import OpenAI as LangchainOpenAI
import logging
from transformers import pipeline
import numpy as np
import statsmodels.api as sm
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

c:\Users\CQTF47\Desktop\IU Masters\Thesis\Project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
OPENAI_API_KEY = "3a6b230b917b4893a150f0ad7fa126cf"
os.environ["AZURE_OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://cpe-clx-openai.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-05-15" 

In [5]:
llm = AzureChatOpenAI(
    model="cpe-clx-gpt4o",
    azure_deployment="cpe-clx-gpt4o",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

embedding_model = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    # deployment_name="cpe-clx-embedding",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"] ,
    azure_deployment="cpe-clx-embedding"
)


In [6]:
qdrant_client = QdrantClient(path="financial_analysis")

In [7]:
collection_name = "financial_documents"

In [8]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='financial_documents')])

In [9]:
if qdrant_client.collection_exists(collection_name=collection_name):
    qdrant_client.delete_collection(collection_name=collection_name)

In [10]:
qdrant_client.get_collections()

CollectionsResponse(collections=[])

In [11]:
qdrant_client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=1536, distance=Distance.COSINE)  # Assuming vector size of 1536
        )

True

In [12]:
folder_path = "C:\\Users\\CQTF47\\Desktop\\IU Masters\\Thesis\\Data\\ADS.DE"

folder_path.split("\\")[-1]

'ADS.DE'

In [13]:
uploaded_folder_name = os.path.basename(folder_path)
print(uploaded_folder_name)

ADS.DE


In [17]:
from pdfminer.high_level import extract_text
import fitz
import io
from PIL import Image
import os
import time

In [18]:
folder_path = "C:\\Users\\CQTF47\\Desktop\\IU Masters\\Thesis\\Data\\ADS.DE"

company_name = os.path.basename(folder_path)

pdf_text = []

logging.info(f"Processing documents for {company_name}")
if os.path.isdir(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            full_path = os.path.join(folder_path, filename)
            pdf_text.append(str(extract_text(full_path)))
    # loader = DirectoryLoader(folder_path, recursive=True)
    # documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(pdf_text)

    vector_store = Qdrant.from_documents(
        documents=docs,
        embedding=embedding_model,
        client=qdrant_client,
        collection_name=collection_name
    )

AttributeError: 'str' object has no attribute 'page_content'

In [4]:
vector_store = Qdrant(
            client=qdrant_client,
            collection_name=collection_name,
            embeddings=embedding_model
        )


c:\Users\CQTF47\Desktop\IU Masters\Thesis\Project\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [9]:
collection_info = qdrant_client.get_collection(collection_name)

print(f"Number of vectors: {collection_info.vectors_count}")

Number of vectors: None


In [5]:
retriever = vector_store.as_retriever()

In [6]:
question = "What is the price of Adidas stock?"

In [7]:
retriever.get_relevant_documents(question)

c:\Users\CQTF47\Desktop\IU Masters\Thesis\Project\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


ValidationError: 1 validation error for Document
page_content
  none is not an allowed value (type=type_error.none.not_allowed)